# Tweets abnehmen und Sentiment analyse anwenden

Die Tweets aus dem Producer werden eingelesen und als PArquet abgelegt.
Darauf wird via *textblob* eine Sentiment analyse gemacht und jedem Tweet ein Rating zugewiesen.

In [0]:
!pip install textblob

In [0]:
# Libraries einlesen

from pyspark.sql import SparkSession
from pyspark.sql.functions import *   # including udf
from pyspark.sql.functions import desc
from pyspark.sql.functions import timestamp_seconds
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql.types import *   
from pyspark.sql import functions as F
from textblob import TextBlob
from datetime import datetime
import random


In [0]:
# Definition der Textklassification via Textblob

@udf
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity

@udf
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity


In [0]:
# nur für Test: erzeugen eines Dummy Preises für Bitcoint

@udf
def bitcoint_price():
    x = 30000 + random.randrange(100, 1000, 2) 
    return x


In [0]:
# Erstellen eines local/private StreamingContext (SparkContext 'sc' besteht in databricks bereits)

ssc = StreamingContext(sc, 2)   # batch interval = 2
stream = ssc.socketTextStream("localhost", 9997)


'''# Ausgabe des stream in die Konsole für 3 Minuten, danach Abbruch
stream.pprint()
try:
  ssc.start()                             
  ssc.awaitTerminationOrTimeout(180)  # Ausgabe im consumer erst nach timeout möglich (sekunden)
finally:
  ssc.stop(False)
'''

Out[4]: '# Ausgabe des stream in die Konsole für 3 Minuten, danach Abbruch\nstream.pprint()\ntry:\n ssc.start() \n ssc.awaitTerminationOrTimeout(180) # Ausgabe im consumer erst nach timeout möglich (sekunden)\nfinally:\n ssc.stop(False)\n'

In [0]:
# Funktionen um Dstream in Dataframe mit 5-Sekunden-Fenster zu transformieren

# Dstream abnehmen
lines = spark \
        .readStream.format("socket") \
        .option("host", "localhost") \
        .option("port", 9997) \
        .load()

# Aufsplitten des Streams in Haupttweet und Re-Tweets
structuredStream = lines \
        .select(split(lines.value, "_t_end_")[0].alias("text") \
                , split(lines.value, "_t_end_")[1].alias("rt_text") \
                , split(lines.value, "_t_end_")[2].alias("rt_text2")
               )

# Timestamps einfügen
now = datetime.now()
structuredStream = structuredStream.withColumn("tweettime", lit(str(now)[:19]))
# structuredStream = structuredStream.withColumn("timestamp", lit(str(now)).cast('timestamp'))

# Sentiment einfügen
structuredStream = structuredStream.withColumn("subjectivity", subjectivity_detection("text").cast('float'))
structuredStream = structuredStream.withColumn("polarity", polarity_detection("text").cast('float'))

# Preis Einfügen
structuredStream = structuredStream.withColumn("price", bitcoint_price().cast('float'))

# Erstellen eines 6 Sekunden-Fensters (als Basis für alle Analysen)
windowedStream = structuredStream \
        .groupBy(window("tweettime", "6 seconds", "6 seconds"))

# Aggregationsfunktion
aggregationsStream = windowedStream \
        .agg(count('tweettime').alias("count_tweets") \
           , avg('subjectivity').alias('sub_avg') \
           , avg('polarity').alias('pol_avg') \
           , avg('price').alias('price_avg')
           )


In [0]:
# Anzeige des aggregierten Streams 
display(aggregationsStream.sort(desc("window.start")))


window,count_tweets,sub_avg,pol_avg,price_avg
"List(2021-05-24T11:54:18.000+0000, 2021-05-24T11:54:24.000+0000)",1241,0.07504316427768189,0.04111812520084796,30551.147461724417


ab hier funktioniert es nicht mehr:
-> der writestream will nicht starten: Watermark fehlt / oder dann writestream kennt kein watermark

In [0]:
# Sink der Daten in ein Parquet file
# dieser SCH.. kommt nicht zum laufen...
# watermark muss definiert werden - aber wie und wo?

query = aggregationsStream \
    .withWatermark('timestamp', '10 seconds') \
    .writeStream \
    .queryName("bc_table") \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "dbfs:/FileStore/bd_project") \
    .option("checkpointLocation", "./check") \
    .trigger(processingTime='30 seconds') \
    .start()

query.awaitTermination()

# Ab hier kein aktiver Code mehr

Zusätzliche Funktionen / variationen von Code / file handling / etc...

In [0]:
'''
# Erweitertes preprocessing 
# könnten wir oben bei der erstellung des lines einsetzen
# putzt zusätzlich noch spezial character raus

def preprocessing(lines):
    words = lines.select(explode(split(lines.value, " _t_end_ ")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words
'''

File System functions

In [0]:
# Listen (Kommentar entfernen damit es funktioniert)
%fs ls dbfs:/FileStore/

In [0]:
# Files und Folders rekursiv löschen (Kommentar entfernen damit es funktioniert)
%fs rm -r dbfs:/FileStore/import-stage/

In [0]:
# Folder erstellen
# dbutils.fs.mkdirs("dbfs/FileStore/bd_project/test")

In [0]:
# Directory anzeigen
# dbutils.fs.ls("dbfs:/dbfs")